<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/main/Predicting_on_unlabeled_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def clean_text(sentences):
  cleaned_sentence = ""
  sentences = sentences.lower().split()
  ##removing stop words
  words = [i for i in sentences if i not in stopwords.words('english')]
  words = " ".join(words)               ##joining our words back to sentences
  return words

In [48]:
def preprocess(length, ngram, d):
  X = df['selftext'][0:length].tolist()
  Y = df['labels'][0:length].tolist()
  for i, post in enumerate(X):
    X[i] = clean_text(post)
  if ngram == "binary":
    vectorizer = CountVectorizer(vocabulary = d, binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer(vocabulary = d)
  else:
    vectorizer = CountVectorizer(vocabulary = d, ngram_range = (2, 2), binary=False) #Bigram/binary
  bag = vectorizer.fit_transform(X)
  #len(bag) == 100 - vectors of posts
  bag = bag.toarray()
  skf = StratifiedKFold(n_splits=10)
  skf.get_n_splits(bag, Y)
  X_train, y_train, X_test, y_test = [], [], [], []
  for i, (train_index, test_index) in enumerate(skf.split(bag, Y)):
    xtrain, ytrain, xtest, ytest = [], [], [], []
    for j in train_index:
      xtrain.append(bag[j])
      ytrain.append(Y[j])
    for j in test_index:
      xtest.append(bag[j])
      ytest.append(Y[j])
    X_train.append(xtrain)
    y_train.append(ytrain)
    X_test.append(xtest)
    y_test.append(ytest)
  return X_train, y_train, X_test, y_test
  #for 100 posts:
  #len(X_train, y_train) == 10, each of the 10 folds have 90 np vector arrays (posts)
  #len(X_test, y_test) == 10, 10 np arrays per fold - 100 posts total
# print(X_test)
# print(y_test)
# print(len(X_test))
# print(X_test[0])
# print(len(X_test[0]))

In [49]:
def classify(X_train, y_train, X_test, y_test, length, count, label):
  w = {0:count, 1:length-count}
  lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter = 1000000, class_weight = w)

  # Use metrics.accuracy_score to measure the score
  #'macro' favors minority, 'weighted' favors majority, 'micro' favors none (when you have multiclass)
  scores = []
  for i in range(0, 10):
    #Get 9 folds for training
    xtrain = X_train[i]
    ytrain = y_train[i]
    lr.fit(xtrain, ytrain)

    xtest = X_test[i]
    ytest = y_test[i]
    y_predict = lr.predict(xtest)
    if label in [1.0, 4.0]:
      score = f1_score(ytest, y_predict, labels=np.unique(y_predict))
    else:
      score = f1_score(ytest, y_predict, labels=np.unique(y_predict))
    scores.append("%.3f" %score)
  return lr, scores

In [ ]:
#Making dictionary to put as the argument for the "vocabulary" parameter in CountVectorizer - need this to get words that the model has not yet seen
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
d=[]
count = 0
for i in df['selftext']:
  for word in i.split():
    if word not in d:
      d.append(word)
print(len(d)) #41716

In [51]:
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
for i in [2.0]:
  count = 0
  for ind in df.index:
    if df.loc[ind, "labels"] != i:
      df.loc[ind, "labels"] = 0
    else:
      df.loc[ind, "labels"] = 1
      count += 1
  length = 3000
  unigram, bigram, binary = [], [], []
  while length < 3100:
    X_train, y_train, X_test, y_test = preprocess(length, "unigram", d)
    lr1, unigram = classify(X_train, y_train, X_test, y_test, length, count, i)
    X_train, y_train, X_test, y_test = preprocess(length, "bigram", d)
    lr2, bigram = classify(X_train, y_train, X_test, y_test, length, count, i)
    X_train, y_train, X_test, y_test = preprocess(length, "binary", d)
    lr3, binary = classify(X_train, y_train, X_test, y_test, length, count, i)
    length += 500
  print(str(length) + " posts: ")
  print("Scores for", i, ": \n")
  sum = 0
  for i in unigram:
      sum += float(i)
  print("Average: " + str(sum/10))
  print("-----------------------")
  sum = 0
  for i in bigram:
    sum += float(i)
  print("Average: " + str(sum/10))
  print("-----------------------")
  sum = 0
  for i in binary:
    sum += float(i)
  print("Average: " + str(sum/10))
  print("\n")
  df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


3500 posts: 
Scores for 2.0 : 

Average: 0.46609999999999996
-----------------------
Average: 0.0
-----------------------
Average: 0.42279999999999995




In [9]:
def a(ngram):
  vectorizer = 0
  if ngram == "binary":
    vectorizer = CountVectorizer(vocabulary=d, binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer(vocabulary=d)
  else: #bigram
    vectorizer = CountVectorizer(vocabulary=d, ngram_range = (2, 2))
  return vectorizer

def b(vectorizer):
  X = np.array(df['selftext'][3100:7751].tolist())
  for i, post in enumerate(X):
    X[i] = clean_text(post)

  bag = vectorizer.fit_transform(X)
  bag = bag.toarray()
  return bag

count = 0
i = 2.0
for ind in df.index:
  if df.loc[ind, "labels"] != i:
      df.loc[ind, "labels"] = 0
  else:
    df.loc[ind, "labels"] = 1
    count += 1

In [15]:
#Getting number of advice posts (1.0 for advice, 0.0 for anything else)
vectorizer = a("unigram")
data = b(vectorizer)
res1 = lr1.predict(data)
print(np.count_nonzero(res1 == 1.0))

vectorizer = a("bigram")
data = b(vectorizer)
res2 = lr2.predict(data)
print(np.count_nonzero(res2 == 1.0))

vectorizer = a("binary")
data = b(vectorizer)
res3 = lr3.predict(data)
print(np.count_nonzero(res3 == 1.0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


64


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


0


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


34


In [ ]:
X = np.array(df['selftext'][501:3500].tolist())
bag = vectorizer.fit_transform(X)
bag = bag.toarray()
y_predict = lr2.predict(bag)
ytest = np.array(df['labels'][501:3500].tolist())
score = f1_score(ytest, y_predict, labels=np.unique(y_predict))
print(score)

In [58]:
X = np.array(df['selftext'][1:100].tolist())
vectorizer = CountVectorizer()
bag = vectorizer.fit_transform(X)
print(bag.toarray()[0])

[0 0 0 ... 0 0 0]
